<a href="https://colab.research.google.com/github/ne1baf/Flaubert/blob/master/hotshot_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Installs

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.0 MB/s eta 0:00:00


In [ ]:
import os
import glob
import pickle
import gzip
import re
import difflib
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from tqdm import tqdm

# Code

In [ ]:
!unzip Archive.zip

Archive:  Archive.zip
  inflating: data_raw.csv            
   creating: menus/
  inflating: menus/_ca_store_infinity-liquor_X2eOGlUtUSyuFVY9NkXT2Q_menu.pkl.gz  
  inflating: menus/_ca_store_legacy-liquor_mDN7s6xsR8mHcmQJacWXZQ_menu.pkl.gz  
  inflating: menus/.DS_Store         
  inflating: __MACOSX/menus/._.DS_Store  
  inflating: menus/_ca_store_cap-liquor_FESxdzInQ76IkyLBI3i3hQ_menu.pkl.gz  
  inflating: menus/_ca_store_hotshot-liquor-store_8SJJxrn3TOuk1yss2RZXSA_menu.pkl.gz  
  inflating: __MACOSX/menus/.__ca_store_hotshot-liquor-store_8SJJxrn3TOuk1yss2RZXSA_menu.pkl.gz  
  inflating: menus/_ca_store_daves-liquor-store-license-%23776739-16th-ave-nw_PE-gPthbQnCxJuCKdhA7PQ_menu.pkl.gz  
  inflating: menus/_ca_store_bk-liquor-store-westbrook_NmLY9lTOQSeIqtms-KCW2g_menu.pkl.gz  
  inflating: categories.json         


In [ ]:
# def lookup_price(item_name):
#     return drink_prices.get(item_name)


# def compare_names(item1, item2):
#     return fuzz.ratio(item1, item2) > 80


# store_info_file = "data_raw.csv"
# store_info_df = pd.read_csv(store_info_file)

# folder_path = "menus"

# store_link_to_compare = "/ca/store/hotshot-liquor-store/8SJJxrn3TOuk1yss2RZXSA"

# store_item_list = []
# store_name = None
# for index, row in tqdm(store_info_df.iterrows(), position=0, leave=True):
#     link = row["Link"]
#     if link == store_link_to_compare:
#         store_name = row["UberStoreName"]
#         file_name = link.replace("/", "_") + "_menu.pkl.gz"
#         file_path = os.path.join(folder_path, file_name)
#         try:
#             with gzip.open(file_path, "rb") as file:
#                 store_menu = pickle.load(file)
#                 store_item_list = [item for category in store_menu for item in category['items']]
#                 break
#         except (FileNotFoundError, pickle.UnpicklingError) as e:
#             print(f"Error loading file: {file_path}. Skipping.")

# if not store_item_list:
#     print(f"Store with link '{store_link_to_compare}' not found in the menu files.")
#     exit()

# drink_prices = {}
# for item in store_item_list:
#     item_name = item['name'].lower()
#     item_price = 0
#     if item['price']:
#         numeric_value = ''.join(c for c in item['price'] if c.isdigit() or c == '.')
#         item_price = float(numeric_value)
#     drink_prices[item_name] = item_price

# target_df = pd.DataFrame(drink_prices.items(), columns=["Item Name", "Target Store Price"])
# target_df['Other Store Prices'] = [list() for _ in range(len(target_df))]
# target_df['Other Store Items'] = [list() for _ in range(len(target_df))]

# comparison_data = []
# for index, row in tqdm(store_info_df.iterrows()):
#     link = row["Link"]
#     if link != store_link_to_compare:
#         other_name = row["UberStoreName"]
#         file_name = link.replace("/", "_") + "_menu.pkl.gz"
#         file_path = os.path.join(folder_path, file_name)
#         try:
#             with gzip.open(file_path, "rb") as file:
#                 other_menu = pickle.load(file)
#                 other_item_list = [item for category in other_menu for item in category['items']]
#                 for other_item in other_item_list:
#                     other_item_name = other_item['name'].lower()
#                     other_price = 0
#                     if other_item['price']:
#                         numeric_value = ''.join(c for c in other_item['price'] if c.isdigit() or c == '.')
#                         other_price = float(numeric_value)
#                     for _, target_row in target_df.iterrows():
#                         if abs(target_row["Target Store Price"] - other_price) <= 5 and compare_names(target_row["Item Name"], other_item_name):
#                             other_store_item = {
#                                 "price": other_price,
#                                 "name": other_item_name,
#                                 "store": other_name
#                             }
#                             target_row["Other Store Prices"].append(other_price)
#                             target_row["Other Store Items"].append(other_store_item)
#                             break
#         except (FileNotFoundError, pickle.UnpicklingError) as e:
#             print(f"Error loading file: {file_path}. Skipping.")

# target_df["Other Store Prices"] = target_df["Other Store Prices"].apply(lambda x: sorted(x))

# comparison_data = []
# for index, row in store_info_df.iterrows():
#     link = row["Link"]
#     if link != store_link_to_compare:
#         other_name = row["UberStoreName"]
#         file_name = link.replace("/", "_") + "_menu.pkl.gz"
#         file_path = os.path.join(folder_path, file_name)
#         try:
#             with gzip.open(file_path, "rb") as file:
#                 other_menu = pickle.load(file)
#                 other_item_list = [item for category in other_menu for item in category['items']]
#                 num_matching_items = sum(1 for item in other_item_list if any(compare_names(item['name'].lower(), target_item) for target_item in drink_prices.keys()))
#                 comparison_data.append((other_name, num_matching_items))
#         except (FileNotFoundError, pickle.UnpicklingError) as e:
#             print(f"Error loading file: {file_path}. Skipping.")

# competitor_df = pd.DataFrame(comparison_data, columns=["Store Name", "Number of Matching Items"])
# # print(target_df)
# # print(competitor_df)

In [ ]:
# Helper function to extract ABV from a string
def extract_abv(item_name):
    match = re.search(r'(\d+(\.\d+)?)% abv', item_name, re.IGNORECASE)
    return float(match.group(1)) if match else None

# Modified comparison function
def compare_names(item1, item2):
    # Base comparison on name similarity
    is_similar = fuzz.ratio(item1, item2) > 80
    # Check ABV consistency
    abv1 = extract_abv(item1)
    abv2 = extract_abv(item2)
    is_abv_consistent = (abv1 == abv2) or (abv1 is None and abv2 is None)
    return is_similar and is_abv_consistent

store_info_file = "data_raw.csv"
store_info_df = pd.read_csv(store_info_file)

folder_path = "menus"

store_link_to_compare = "/ca/store/hotshot-liquor-store/8SJJxrn3TOuk1yss2RZXSA"

store_item_list = []
store_name = None

# Load store items
for index, row in tqdm(store_info_df.iterrows(), position=0, leave=True):
    link = row["Link"]
    if link == store_link_to_compare:
        store_name = row["UberStoreName"]
        file_name = link.replace("/", "_") + "_menu.pkl.gz"
        file_path = os.path.join(folder_path, file_name)
        try:
            with gzip.open(file_path, "rb") as file:
                store_menu = pickle.load(file)
                store_item_list = [item for category in store_menu for item in category['items']]
                break
        except (FileNotFoundError, pickle.UnpicklingError) as e:
            print(f"Error loading file: {file_path}. Skipping.")

if not store_item_list:
    print(f"Store with link '{store_link_to_compare}' not found in the menu files.")
    exit()

# Extract drink prices
drink_prices = {}
for item in store_item_list:
    item_name = item['name'].lower()
    item_price = 0
    if item['price']:
        numeric_value = ''.join(c for c in item['price'] if c.isdigit() or c == '.')
        item_price = float(numeric_value)
    drink_prices[item_name] = item_price

# Create target DataFrame
target_df = pd.DataFrame(drink_prices.items(), columns=["Item Name", "Target Store Price"])
target_df['Other Store Prices'] = [list() for _ in range(len(target_df))]
target_df['Other Store Items'] = [list() for _ in range(len(target_df))]

# Compare with other stores
for index, row in tqdm(store_info_df.iterrows()):
    link = row["Link"]
    if link != store_link_to_compare:
        other_name = row["UberStoreName"]
        file_name = link.replace("/", "_") + "_menu.pkl.gz"
        file_path = os.path.join(folder_path, file_name)
        try:
            with gzip.open(file_path, "rb") as file:
                other_menu = pickle.load(file)
                other_item_list = [item for category in other_menu for item in category['items']]
                store_best_matches = {}
                for other_item in other_item_list:
                    other_item_name = other_item['name'].lower()
                    other_price = 0
                    if other_item['price']:
                        numeric_value = ''.join(c for c in other_item['price'] if c.isdigit() or c == '.')
                        other_price = float(numeric_value)
                    for _, target_row in target_df.iterrows():
                        if abs(target_row["Target Store Price"] - other_price) <= 5:
                            similarity_score = fuzz.ratio(target_row["Item Name"], other_item_name)
                            if similarity_score > 80:
                                key = (target_row["Item Name"], other_name)
                                if key not in store_best_matches or similarity_score > store_best_matches[key]['score']:
                                    store_best_matches[key] = {
                                        'score': similarity_score,
                                        'other_item': {
                                            "price": other_price,
                                            "name": other_item_name,
                                            "store": other_name
                                        }
                                    }
                for key, value in store_best_matches.items():
                    target_item_name, _ = key
                    target_df.loc[target_df["Item Name"] == target_item_name, "Other Store Prices"].apply(lambda x: x.append(value['other_item']['price']))
                    target_df.loc[target_df["Item Name"] == target_item_name, "Other Store Items"].apply(lambda x: x.append(value['other_item']))
        except (FileNotFoundError, pickle.UnpicklingError) as e:
            print(f"Error loading file: {file_path}. Skipping.")

target_df["Other Store Prices"] = target_df["Other Store Prices"].apply(lambda x: sorted(x))

comparison_data = []
for index, row in store_info_df.iterrows():
    link = row["Link"]
    if link != store_link_to_compare:
        other_name = row["UberStoreName"]
        file_name = link.replace("/", "_") + "_menu.pkl.gz"
        file_path = os.path.join(folder_path, file_name)
        try:
            with gzip.open(file_path, "rb") as file:
                other_menu = pickle.load(file)
                other_item_list = [item for category in other_menu for item in category['items']]
                num_matching_items = sum(1 for item in other_item_list if any(compare_names(item['name'].lower(), target_item) for target_item in drink_prices.keys()))
                comparison_data.append((other_name, num_matching_items))
        except (FileNotFoundError, pickle.UnpicklingError) as e:
            print(f"Error loading file: {file_path}. Skipping.")

competitor_df = pd.DataFrame(comparison_data, columns=["Store Name", "Number of Matching Items"])


0it [00:00, ?it/s]
6it [02:27, 24.57s/it]


In [ ]:
target_df[target_df['Other Store Prices'].apply(any)].to_csv("price_comparison.csv")

In [ ]:
df = target_df[target_df['Other Store Items'].apply(any)].copy()

# Add new columns to the DataFrame
df['Cheapest Store'] = None
df['Cheapest Price'] = None
df['Price Difference'] = None
df['Action'] = None
df['New Price'] = None
df['Final Percentage Difference'] = None

# Process each row of the DataFrame
for index, row in df.iterrows():
    other_store_items = row['Other Store Items']
    other_store_prices = [item['price'] for item in other_store_items]
    cheapest_price = min(other_store_prices)
    cheapest_store = next(item['store'] for item in other_store_items if item['price'] == cheapest_price)
    target_price = row['Target Store Price']
    price_diff = target_price - cheapest_price
    final_percentage_diff = (price_diff / cheapest_price) * 100  # Calculate the percentage difference
    if price_diff > 0 or abs(final_percentage_diff) > 5:
        if price_diff > 0:
            action = 'Decrease'
            new_price = cheapest_price * 0.95
        else:
            action = 'Increase'
            new_price = cheapest_price * 1.05
        new_price = round(new_price - 0.01, 1)  # Make sure the price ends in .X9
        final_percentage_diff = ((new_price - cheapest_price) / cheapest_price) * 100  # Recalculate the percentage difference
    else:
        action = 'No Change'
        new_price = target_price
    df.loc[index, 'Cheapest Store'] = cheapest_store
    df.loc[index, 'Cheapest Price'] = cheapest_price
    df.loc[index, 'Price Difference'] = round((price_diff / cheapest_price) * 100, 2)  # Update with percentage difference
    df.loc[index, 'Action'] = action
    df.loc[index, 'New Price'] = new_price
    df.loc[index, 'Final Percentage Difference'] = round(final_percentage_diff, 2)


In [ ]:
# Reorder columns and rename 'Target Store Price'
df = df[['Item Name', 'Target Store Price', 'Cheapest Price', 'Cheapest Store', 'Price Difference', 'Other Store Prices', 'Action', 'New Price', 'Final Percentage Difference', 'Other Store Items']]
df.rename(columns={'Target Store Price': 'Your Store Price'}, inplace=True)
df = df[df['Item Name'] != '•']

In [ ]:
df.to_csv("test.csv")

Formatting [WIP]

In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Color, PatternFill

import pandas as pd
import openpyxl
from openpyxl.styles import GradientFill

from openpyxl.styles import PatternFill

def gradient_formatting(cell, price_difference):
    if price_difference > 0:
        cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")  # red
    else:
        cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # green



In [ ]:
# Ensure 'Price Difference' is of float type
df['Price Difference'] = df['Price Difference'].astype(float)

# Write the DataFrame to an Excel file and apply formatting
with pd.ExcelWriter('output.xlsx', engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    ws = writer.sheets['Sheet1']

    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=1):
        for cell in row:
            idx = cell.row - 2  # Adjusting for 1-indexed rows and header row
            gradient_formatting(cell, df.iloc[idx]['Price Difference'])


## QA

In [ ]:
# Find and display rows with the same "Item Name"
duplicate_items = df[df.duplicated(subset=['Item Name'], keep=False)]

print("Rows with the same Item Name:")
print(duplicate_items)

Rows with the same Item Name:
Empty DataFrame
Columns: [Item Name, Target Store Price, Other Store Prices, Other Store Items, Cheapest Store, Cheapest Price, Price Difference, Action, New Price, Final Percentage Difference]
Index: []


# Code old

In [ ]:
file_path = "hotshot.pkl.gz"

try:
    with gzip.open(file_path, "rb") as file:
        menu_data = pickle.load(file)
        # Process the menu data as needed
        print(menu_data)
except FileNotFoundError:
    print("File not found:", file_path)
except pickle.UnpicklingError:
    print("Error while unpickling the file:", file_path)


[{'name': 'Picked for you', 'items': [{'name': 'Pabst Blue Ribbon 15 pack, 15x355ml (4.9%abv)', 'price': '29.99', 'image_url': 'https://tb-static.uber.com/prod/image-proc/processed_images/8753ab4217cb4b1477aac24b02fbed98/b4facf495c22df52f3ca635379ebe613.jpeg'}, {'name': 'Pabst Blue Ribbon, 24 cans-24*355mL beer (4.90% ABV)', 'price': '44.49', 'image_url': 'https://tb-static.uber.com/prod/image-proc/processed_images/ce2a8c03fe8920688d7b0cba44c3b5c3/b4facf495c22df52f3ca635379ebe613.jpeg'}, {'name': 'White Claw Black Cherry, 6*355mL (5% ABV)', 'price': '19.59', 'image_url': ''}, {'name': 'Corona, 6 bottles-6*330mL imported beer (4.60% ABV)', 'price': '21.99', 'image_url': ''}, {'name': 'Jagermeister, 750mL liqueurs (35% ABV)', 'price': '37.09', 'image_url': ''}]}, {'name': 'BRAND NEW BEERS YOU SHOULD KNOW', 'items': [{'name': 'PHILLIPS PARROT EYES MARGARITA GOSE 6*355ML (4.5%)ABV', 'price': '21.99', 'image_url': ''}, {'name': 'MALIBU PINEAPPLE BAY BREEZE 4*355ML (7%ABV)', 'price': '18.99'

In [ ]:
menu_data

In [ ]:
!unzip Archive.zip

Archive:  Archive.zip
   creating: menus/
  inflating: menus/_ca_store_infinity-liquor_X2eOGlUtUSyuFVY9NkXT2Q_menu.pkl.gz  
  inflating: menus/_ca_store_legacy-liquor_mDN7s6xsR8mHcmQJacWXZQ_menu.pkl.gz  
  inflating: menus/cap.pkl.gz        
  inflating: menus/.DS_Store         
  inflating: __MACOSX/menus/._.DS_Store  
  inflating: menus/_ca_store_cap-liquor_FESxdzInQ76IkyLBI3i3hQ_menu.pkl.gz  
  inflating: menus/test.pkl.gz       
  inflating: menus/_ca_store_daves-liquor-store-license-%23776739-16th-ave-nw_PE-gPthbQnCxJuCKdhA7PQ_menu.pkl.gz  
  inflating: menus/test2.gz          
  inflating: __MACOSX/menus/._test2.gz  
  inflating: menus/_ca_store_bk-liquor-store-westbrook_NmLY9lTOQSeIqtms-KCW2g_menu.pkl.gz  
  inflating: data_raw.csv            
  inflating: categories.json         


In [ ]:
# rm -rf __MACOSX menus

In [ ]:
def lookup_price(item_name):
    return drink_prices.get(item_name)

# Step 1: Read the store information from the CSV file
store_info_file = "data_raw.csv"  # Replace with the actual path to the CSV file
store_info_df = pd.read_csv(store_info_file)

# Step 2: Define the folder path where the menu files are located
folder_path = "menus"  # Replace with the actual folder path

# Step 3: Select the store link you want to compare with others
store_link_to_compare = "/ca/store/hotshot-liquor-store/8SJJxrn3TOuk1yss2RZXSA"  # Replace with the desired store link

# Step 4: Collect the item list for the store you want to compare
store_item_list = None
store_name = None
for index, row in store_info_df.iterrows():
    link = row["Link"]
    if link == store_link_to_compare:
        store_name = row["UberStoreName"]
        file_name = link.replace("/", "_") + "_menu.pkl.gz"
        file_path = os.path.join(folder_path, file_name)
        try:
            with gzip.open(file_path, "rb") as file:
                store_menu = pickle.load(file)
                for category in store_menu:
                    for item in category['items']:
                        item['category'] = category['name']
                store_item_list = [item for category in store_menu for item in category['items']]
                break
        except (FileNotFoundError, pickle.UnpicklingError) as e:
            print(f"Error loading file: {file_path}. Skipping.")

if store_item_list is None:
    print(f"Store with link '{store_link_to_compare}' not found in the menu files.")
    exit()

In [ ]:
# Step 5: Extract drink names and prices from the store's item list
drink_prices = {}
for item in store_item_list:
    name = item['name']
    price = float(item['price']) if item['price'] else 0
    if item['price'] is None:
        print(f"Item : {item} is null")
    drink_prices = {item['name'].lower(): float(item['price']) if item['price'] else 0 for item in store_item_list}

In [ ]:
# Step 6: Compare the selected store with other stores
comparison_data = []
for index, row in tqdm(store_info_df.iterrows()):
    link = row["Link"]
    if link != store_link_to_compare:
        other_name = row["UberStoreName"]
        file_name = link.replace("/", "_") + "_menu.pkl.gz"
        file_path = os.path.join(folder_path, file_name)
        try:
            with gzip.open(file_path, "rb") as file:
                other_menu = pickle.load(file)
                for category in other_menu:
                    for item in category['items']:
                        item['category'] = category['name']
                other_item_list = [{**item, 'name': item['name'].lower()} for category in other_menu for item in category['items']]
                matching_items = []
                for other_item in other_item_list:
                  other_item_name = other_item['name'].lower()
                  other_price = 0
                  if other_item['price']:
                      numeric_value = ''.join(c for c in other_item['price'] if c.isdigit() or c == '.')
                      other_price = float(numeric_value)
                  if row["UberStoreName"] == store_name:
                    continue  # Skip comparing the target store prices against itself
                    matching_name = difflib.get_close_matches(other_item_name, drink_prices.keys(), n=1, cutoff=0.8)
                    if matching_name:
                        matched_price = drink_prices[matching_name[0]]
                        price_diff = abs(other_price - matched_price)
                        if price_diff <= 10:
                            matching_items.append((other_item_name, other_price, matched_price))
                if matching_items:
                    comparison_data.append((other_name, matching_items))
        except (FileNotFoundError, pickle.UnpicklingError) as e:
            print(f"Error loading file: {file_path}. Skipping.")

6it [00:00, 254.17it/s]


In [ ]:
def lookup_price(item_name):
    return drink_prices.get(item_name)

# Step 1: Read the store information from the CSV file
store_info_file = "data_raw.csv"  # Replace with the actual path to the CSV file
store_info_df = pd.read_csv(store_info_file)

# Step 2: Define the folder path where the menu files are located
folder_path = "menus"  # Replace with the actual folder path

# Step 3: Select the store link you want to compare with others
store_link_to_compare = "/ca/store/hotshot-liquor-store/8SJJxrn3TOuk1yss2RZXSA"  # Replace with the desired store link

# Step 4: Collect the item list for the store you want to compare
store_item_list = None
store_name = None
for index, row in store_info_df.iterrows():
    link = row["Link"]
    if link == store_link_to_compare:
        store_name = row["UberStoreName"]
        file_name = link.replace("/", "_") + "_menu.pkl.gz"
        file_path = os.path.join(folder_path, file_name)
        try:
            with gzip.open(file_path, "rb") as file:
                store_menu = pickle.load(file)
                for category in store_menu:
                    for item in category['items']:
                        item['category'] = category['name']
                store_item_list = [item for category in store_menu for item in category['items']]
                break
        except (FileNotFoundError, pickle.UnpicklingError) as e:
            print(f"Error loading file: {file_path}. Skipping.")

if store_item_list is None:
    print(f"Store with link '{store_link_to_compare}' not found in the menu files.")
    exit()

# Step 5: Extract drink names and prices from the store's item list
drink_prices = {}
for item in store_item_list:
    name = item['name']
    price = float(item['price']) if item['price'] else 0
    if item['price'] is None:
        print(f"Item : {item} is null")
    drink_prices = {item['name'].lower(): float(item['price']) if item['price'] else 0 for item in store_item_list}

# Step 6: Compare the selected store with other stores
comparison_data = []
for index, row in tqdm(store_info_df.iterrows()):
    link = row["Link"]
    if link != store_link_to_compare:
        other_name = row["UberStoreName"]
        file_name = link.replace("/", "_") + "_menu.pkl.gz"
        file_path = os.path.join(folder_path, file_name)
        try:
            with gzip.open(file_path, "rb") as file:
                other_menu = pickle.load(file)
                for category in other_menu:
                    for item in category['items']:
                        item['category'] = category['name']
                other_item_list = [{**item, 'name': item['name'].lower()} for category in other_menu for item in category['items']]
                matching_items = []
                for other_item in other_item_list:
                  other_item_name = other_item['name'].lower()
                  other_price = 0
                  if other_item['price']:
                      numeric_value = ''.join(c for c in other_item['price'] if c.isdigit() or c == '.')
                      other_price = float(numeric_value)
                  if row["UberStoreName"] == store_name:
                    continue  # Skip comparing the target store prices against itself
                  matching_name = difflib.get_close_matches(other_item_name, drink_prices.keys(), n=1, cutoff=0.8)
                  if matching_name:
                    print("match")
                    matched_price = drink_prices[matching_name[0]]
                    price_diff = abs(other_price - matched_price)
                    if price_diff <= 10:
                        matching_items.append((other_item_name, other_price, matched_price))
                if matching_items:
                  comparison_data.append((other_name, matching_items))
        except (FileNotFoundError, pickle.UnpicklingError) as e:
            print(f"Error loading file: {file_path}. Skipping.")

# Step 7: Compare and calculate stats for each item
comparison_data = []
for item_name, target_price in tqdm(drink_prices.items()):
    item_data = {"Item Name": item_name, "Target Store Price": target_price}
    other_prices = []
    for index, row in store_info_df.iterrows():
        link = row["Link"]
        other_name = row["UberStoreName"]
        if link != store_link_to_compare:
            file_name = link.replace("/", "_") + "_menu.pkl.gz"
            file_path = os.path.join(folder_path, file_name)
            try:
                with gzip.open(file_path, "rb") as file:
                    other_menu = pickle.load(file)
                    other_item_list = [item for category in other_menu for item in category['items']]
                    other_item = next((item for item in other_item_list if item['name'].lower() == item_name), None)
                    if other_item is not None:
                      other_price = 0
                      if other_item['price']:
                          numeric_value = ''.join(c for c in other_item['price'] if c.isdigit() or c == '.')
                          other_price = float(numeric_value)
                      other_prices.append((other_price, other_name))
            except (FileNotFoundError, pickle.UnpicklingError) as e:
                print(f"Error loading file: {file_path}. Skipping.")

    if other_prices:
        other_prices.sort()
        item_data["Other Stores Average Price"] = sum(price for price, _ in other_prices) / len(other_prices)
        item_data["Other Stores Lowest Price"] = other_prices[0][0]
        item_data["Other Stores Highest Price"] = other_prices[-1][0]
        item_data["Store With Lowest Price"] = other_prices[0][1]
        item_data["Store With Highest Price"] = other_prices[-1][1]
        comparison_data.append(item_data)

# Step 8: Create a dataframe from the comparison data
df = pd.DataFrame(comparison_data)


In [ ]:
df.to_csv('price_diff_recap.csv')

In [ ]:
comparison_data = []
for index, row in tqdm(store_info_df.iterrows()):
    link = row["Link"]
    if link != store_link_to_compare:
        other_name = row["UberStoreName"]
        file_name = link.replace("/", "_") + "_menu.pkl.gz"
        file_path = os.path.join(folder_path, file_name)
        try:
            with gzip.open(file_path, "rb") as file:
                other_menu = pickle.load(file)
                other_item_list = [item for category in other_menu for item in category['items']]
                for other_item in other_item_list:
                    other_item_name = other_item['name'].lower()
                    other_price = 0
                    if other_item['price']:
                        numeric_value = ''.join(c for c in other_item['price'] if c.isdigit() or c == '.')
                        other_price = float(numeric_value)
                    for _, target_row in target_df.iterrows():
                        if abs(target_row["Target Store Price"] - other_price) <= 5 and compare_names(target_row["Item Name"], other_item_name):
                            other_store_item = {
                                "price": other_price,
                                "name": other_item_name,
                                "store": other_name
                            }
                            target_row["Other Store Prices"].append(other_price)
                            target_row["Other Store Items"].append(other_store_item)
                            break
        except (FileNotFoundError, pickle.UnpicklingError) as e:
            print(f"Error loading file: {file_path}. Skipping.")

6it [01:40, 16.72s/it]


In [ ]:
import pandas as pd

# Create a list to store the data for the DataFrame
data = []

# Iterate through the comparison_data list
for store_name, products in comparison_data:
    # Iterate through the products list
    for product_name, price_first_store, price_other_store in products:
        # Add the product data to the data list
        data.append([store_name, product_name, price_first_store, price_other_store, price_first_store - price_other_store])

# Create a DataFrame from the data list
df = pd.DataFrame(data, columns=['Store_Name', 'Product', 'Price_First_Store', 'Price_Other_Store', 'Price_Difference'])

# Filter out the products that are cheaper at the other store
cheaper_at_other_store_df = df[df['Price_Difference'] > 0]

# Sort the DataFrame by the difference in price, in descending order
cheaper_at_other_store_df = cheaper_at_other_store_df.sort_values('Price_Difference', ascending=False)

# Display the DataFrame
cheaper_at_other_store_df

,Store_Name,Product,Price_First_Store,Price_Other_Store,Price_Difference


# other

In [ ]:
df2 = pd.read_csv('data_raw.csv')
df1 = competitor_df.copy()

# We assume df1 is already loaded into the environment.

# Use pandas merge to combine the two dataframes on the store names.
# By default, pandas merge uses an 'inner' join, which means only the store names that exist in both dataframes will be kept in the merged dataframe.
df = df2.merge(df1, left_on='UberStoreName', right_on='Store Name')

# If you want to keep all rows in df2 and only add matching rows from df1 (similar to a left join in SQL), you can specify `how='left'` in the merge function.
# df = df2.merge(df1, left_on='UberStoreName', right_on='Store Name', how='left')


In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
df[df['Number of Matching Items'] > 10].drop_duplicates().to_csv('competitors.csv')

# Tests

In [ ]:
import pickle
import gzip

In [ ]:
file_path = "test.pkl.gz"

try:
    with gzip.open(file_path, "rb") as file:
        menu_data = pickle.load(file)
        # Process the menu data as needed
        print(menu_data)
except FileNotFoundError:
    print("File not found:", file_path)
except pickle.UnpicklingError:
    print("Error while unpickling the file:", file_path)

[{'name': 'All High Rated Wines', 'items': [{'name': "The Prisoner' Red Napa Valley (USA),750ml,(ABV-15.20%)", 'price': '72.99', 'image_url': ''}, {'name': 'Whitehaven Sauv Blanc (NZ),750ML,(ABV-13%)', 'price': '24.99', 'image_url': ''}, {'name': 'Penfolds Bin 28 Kalimna Shiraz (Australia),750ml,(ABV-14.50%)', 'price': '59.99', 'image_url': 'https://tb-static.uber.com/prod/image-proc/processed_images/bba81cc3566218ba397d523a7e2f518b/5954bcb006b10dbfd0bc160f6370faf3.jpeg'}, {'name': 'J.Lohr HILLTOP Cab Sauv, USA,750ML(ABV-14.90%)', 'price': '65.99', 'image_url': ''}, {'name': 'Whispering Angel Provence Rose (France),750ml,(ABV-13%)', 'price': '39.99', 'image_url': 'https://tb-static.uber.com/prod/image-proc/processed_images/ce9ee03dfb47936eecf58aa98b934fd2/5954bcb006b10dbfd0bc160f6370faf3.jpeg'}, {'name': 'Gray Monk Gewurztraminer (CA),750ml,(ABV-12.30%)', 'price': '25.49', 'image_url': 'https://tb-static.uber.com/prod/image-proc/processed_images/48faac6cf05e43c32da90864cd3bddc4/5954bcb